# 🔌 Week 09-10 · Notebook 05 · MLP Implementation for Sensor Fusion

Blend tabular sensor data with textual maintenance annotations to predict downtime risk.

## 🎯 Learning Objectives
- Build a PyTorch MLP that ingests fused sensor + text embedding features.
- Engineer weighted loss functions reflecting production cost of downtime.
- Analyze feature importance to drive maintenance prioritization.
- Produce model documentation for cross-functional review.

## 🧩 Scenario
Production planners want an early warning score that combines vibration sensors, temperature readings, and technician notes. False negatives can cause unplanned downtime costing ₹4 lakh per hour.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

torch.manual_seed(2025)

## 📦 Synthetic Sensor + Text Features
- Sensor block: vibration RMS, temperature, humidity, power draw.
- Text block: 8-dim embeddings from prior notebook (placeholder).

In [ ]:
def create_dataset(num_samples=800):
    sensor = np.random.normal(loc=[0.8, 60, 45, 15], scale=[0.2, 8, 12, 4], size=(num_samples, 4))
    text_embed = np.random.normal(size=(num_samples, 8))
    combined = np.concatenate([sensor, text_embed], axis=1)
    risk = (
    labels = (risk > 0.5).astype(np.float32)
    return combined.astype(np.float32), labels.reshape(-1, 1)

features, labels = create_dataset()
features.shape, labels.mean()

In [ ]:
class SensorFusionDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features)
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

dataset = SensorFusionDataset(features, labels)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

## 🧠 MLP Architecture
- Hidden dims (64 → 32) with dropout.
- Output probability indicates high downtime risk in the next 8 hours.

In [ ]:
class FusionMLP(nn.Module):
    def __init__(self, input_dim=12):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(p=0.2)
        self.fc_out = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.fc_out(x))

model = FusionMLP()

## 💸 Cost-Aware Loss Function
Weight false negatives higher to represent lost production hours.

In [ ]:
def cost_weighted_bce(preds, targets):
    fn_penalty = 5.0  # ₹4 lakh/hour equivalent scaling
    epsilon = 1e-7
    loss = -(
    return loss.mean()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train_epoch(loader):
    model.train()
    total_loss = 0.0
    for batch_features, batch_labels in loader: